In [1]:
# ============================================================
# MVP: Classificação por nome das variáveis do Ovation
# Saídas:
#  - coluna "categoria" ∈ {decisao_operacao, resultado, proxi_direta}
#  - coluna "wr_wm_role" ∈ {nu, delta, tau, o2, recirc, loop_seal, coal_flow, pressure, temp, dp, gas_flow, unknown}
#  - coluna "rationale" explicando a regra aplicada
#  - arquivo "ovation_mapeado_nome_unidade_classificado.csv"
# ============================================================

import pandas as pd
import re
import unicodedata
from pathlib import Path

# ------------------------------------------------------------
# Configuração: paths de entrada/saída
# Ajuste se necessário
# ------------------------------------------------------------
INPUT_PATH = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_mapeado_nome_unidade.csv")
OUTPUT_PATH = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_mapeado_nome_unidade_classificado.csv")

# ------------------------------------------------------------
# Utilitários
# ------------------------------------------------------------
def normalize_text(s: str) -> str:
    if pd.isna(s):
        return ""
    s = str(s)
    s = unicodedata.normalize("NFKD", s).encode("ASCII", "ignore").decode("ASCII")
    s = s.lower().strip()
    s = re.sub(r"[^a-z0-9_\-\. ]+", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s

def any_pattern(name: str, patterns) -> bool:
    """Retorna True se qualquer padrão regex de 'patterns' casar em 'name'."""
    return any(re.search(p, name) for p in patterns)

# ------------------------------------------------------------
# Heurísticas de classificação por nome
# Edite conforme a sua taxonomia de tags do Ovation
# ------------------------------------------------------------
RULES = {
    # Variáveis de decisão (operador/controle ajusta diretamente)
    "decisao_operacao": {
        "patterns": [
            r"\b(sp|set[_ ]?point)\b",      # setpoints em geral
            r"\bvalv(e|ula).*open",         # abertura de válvulas
            r"\b(feeder|belt).*speed\b",    # velocidade alimentador/correia
            r"\b(belt).*coal\b",            # fluxo carvão na correia
            r"\b(primary|secondary).*air\b",# ar primário/ secundário
            r"\bpa\b", r"\bsa\b",           # siglas comuns PA/SA
            r"\bseal.*air\b",               # aeração do loop seal
            r"\bloop[_ ]?seal\b",           # loop seal explícito
            r"\brecirc(ulation)?\b",        # comandos de recirculação
            r"\bfan.*(speed|cmd|sp)\b",     # ventiladores com comando
            r"\b(o2|oxygen).*(sp|set[_ ]?point)\b"
        ],
        "rationale": "nome sugere variável de comando ou setpoint ajustável"
    },

    # Proxies diretas do desgaste (usaremos como rótulo se confiáveis)
    # Itens típicos: espessura/UT, erosão, alarmes/tags de desgaste, temperatura de pele do tubo, eventos de falha
    "proxi_direta": {
        "patterns": [
            r"\b(erosion|wear|desgaste)\b",
            r"\b(thickness|espessura|ut)\b",
            r"\b(tube|skin).*temp\b",           # temperatura de pele do tubo (pode refletir fouling/erosao local)
            r"\b(leak|vazamento)\b",
            r"\b(hotspot)\b",
            r"\b(repair|reparo|manutencao)\b",
            r"\b(alarm|trip).*eros",            # alarmes de erosão
        ],
        "rationale": "nome sugere medição/alarme diretamente ligada a desgaste/falha"
    },

    # Variáveis de resultado (estado do processo, não diretamente comandadas)
    "resultado": {
        "patterns": [
            r"\bpress(ure)?\b",                 # pressões gerais
            r"\btemp(erature)?\b",              # temperaturas gerais
            r"\bdp\b|\bdelta[_ ]?p\b",          # queda de pressão
            r"\bo2\b|\boxygen\b(?!.*sp)",       # O2 medido (sem SP)
            r"\bco\b(?![a-z])",                 # CO medido
            r"\bso2\b|\bnox\b",                 # emissões
            r"\bfurnace\b|\bbackpass\b|\bbed\b|\bcyclone\b",
            r"\bgas.*flow\b(?!.*sp)",           # vazão de gás medida
            r"\bmetal.*temp\b(?!.*sp)",         # temp. metal medida (cuidado: pode ser proxy direta; tratar por prioridade abaixo)
        ],
        "rationale": "nome sugere medição de estado/desempenho do processo"
    },
}

# ------------------------------------------------------------
# Mapeamento preliminar para papéis físicos Wr/Wm
# Isso NÃO classifica categoria; apenas sugere o papel físico provável
# ------------------------------------------------------------
WR_WM_MAP = [
    (r"\b(delta[_ ]?p|dp)\b|\bpress(ure)? drop\b", "dp"),
    (r"\b(o2|oxygen)\b", "o2"),
    (r"\b(temp|temperature)\b", "temp"),        # candidato a 'tau' por zona
    (r"\bbackpass\b", "temp"),                  # ajuda a identificar zona/temperatura
    (r"\bbed\b", "temp"),
    (r"\b(primary|secondary).*air\b", "gas_flow"),
    (r"\bgas.*flow\b", "gas_flow"),
    (r"\brecirc(ulation)?\b", "recirc"),
    (r"\bseal.*air\b|\bloop[_ ]?seal\b", "loop_seal"),
    (r"\b(belt|feeder).*coal\b|\bcoal.*flow\b", "coal_flow"),
    # ν e δ são difíceis de medir diretamente; geralmente ν ~ função de fluxos/DP e δ ~ granulometria
    # Use 'dp' + 'gas_flow' como proxies de ν; δ provavelmente ficará 'unknown' salvo se houver "particle size" no nome
    (r"\bparticle.*size\b|\bgranulo", "delta"),
]

def suggest_wr_wm_role(name_norm: str) -> str:
    for pat, role in WR_WM_MAP:
        if re.search(pat, name_norm):
            return role
    return "unknown"

# ------------------------------------------------------------
# Estratégia de classificação com prioridade
# 1) proxi_direta tem prioridade máxima (se casar, é próxi e pronto)
# 2) decisao_operacao
# 3) resultado
# Caso nada case: 'resultado' por segurança, ou 'desconhecida'
# ------------------------------------------------------------
def classify_category(name_norm: str):
    # proxi_direta primeiro
    if any_pattern(name_norm, RULES["proxi_direta"]["patterns"]):
        return "proxi_direta", RULES["proxi_direta"]["rationale"]
    # depois decisao_operacao
    if any_pattern(name_norm, RULES["decisao_operacao"]["patterns"]):
        return "decisao_operacao", RULES["decisao_operacao"]["rationale"]
    # depois resultado
    if any_pattern(name_norm, RULES["resultado"]["patterns"]):
        return "resultado", RULES["resultado"]["rationale"]
    # fallback
    return "resultado", "fallback: sem padrão explícito; assumido como medição de resultado"

# ------------------------------------------------------------
# Leitura do CSV e aplicação
# Espera-se que o arquivo tenha ao menos uma coluna com o "nome" da variável.
# Ajuste 'COL_NAME' para a coluna correta se necessário.
# ------------------------------------------------------------
df = pd.read_csv(INPUT_PATH)

# Tente detectar automaticamente a coluna de "nome" e "unidade" mais prováveis
possible_name_cols = [c for c in df.columns if re.search(r"(tag|nome|name|variavel)", c, flags=re.I)]
possible_unit_cols = [c for c in df.columns if re.search(r"(unid|unit|unidade)", c, flags=re.I)]

COL_NAME = possible_name_cols[0] if possible_name_cols else df.columns[0]
COL_UNIT = possible_unit_cols[0] if possible_unit_cols else None

# Normalização do nome
df["_name_norm"] = df[COL_NAME].apply(normalize_text)

# Classificação
cats, rats, roles = [], [], []
for name_norm in df["_name_norm"]:
    categoria, rationale = classify_category(name_norm)
    cats.append(categoria)
    rats.append(rationale)
    roles.append(suggest_wr_wm_role(name_norm))

df["categoria"] = cats
df["rationale"] = rats
df["wr_wm_role"] = roles

# Ordenação para inspeção
order = pd.CategoricalDtype(categories=["decisao_operacao", "resultado", "proxi_direta"], ordered=True)
df["categoria"] = df["categoria"].astype(order)
df = df.sort_values(["categoria", "wr_wm_role", COL_NAME]).reset_index(drop=True)

# Salvar
df.to_csv(OUTPUT_PATH, index=False)

# Sumário
summary = df.groupby(["categoria", "wr_wm_role"]).size().rename("count").reset_index()
print("Resumo por categoria e papel físico sugerido:")
print(summary.to_string(index=False))

print("\nExemplo de 5 linhas por categoria:")
for cat in ["decisao_operacao", "resultado", "proxi_direta"]:
    sample = df[df["categoria"] == cat].head(5)
    if not sample.empty:
        print(f"\nCategoria: {cat}")
        print(sample[[COL_NAME, "categoria", "wr_wm_role", "rationale"]].to_string(index=False))
    else:
        print(f"\nCategoria: {cat} — sem linhas.")


Resumo por categoria e papel físico sugerido:
       categoria wr_wm_role  count
decisao_operacao  coal_flow      0
decisao_operacao       temp      0
decisao_operacao    unknown      0
       resultado  coal_flow      1
       resultado       temp      1
       resultado    unknown    148
    proxi_direta  coal_flow      0
    proxi_direta       temp      0
    proxi_direta    unknown      0

Exemplo de 5 linhas por categoria:

Categoria: decisao_operacao — sem linhas.

Categoria: resultado
                       point_name categoria wr_wm_role                                                          rationale
(A) COAL-AIR-PERC-FLOW.UNIT1@NET1 resultado  coal_flow fallback: sem padrão explícito; assumido como medição de resultado
     (A) BED-TEMP-AVRG.UNIT1@NET1 resultado       temp               nome sugere medição de estado/desempenho do processo
       (A) 10BBA06XQ01.UNIT1@NET1 resultado    unknown fallback: sem padrão explícito; assumido como medição de resultado
       (A) 10BB

C:\Users\wilso\AppData\Local\Temp\ipykernel_5136\1907681831.py:178: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = df.groupby(["categoria", "wr_wm_role"]).size().rename("count").reset_index()


In [3]:
# ============================================================
# Correção estrutural 2023/2024 + verificação por similaridade
# Saída: ovation_dados_curado_v1.csv
# ============================================================
import pandas as pd
import numpy as np
import re
from pathlib import Path
from datetime import datetime
from typing import Optional

INPUT = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado.csv")
OUTPUT = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v1.csv")

# ----------------------------
# Helpers
# ----------------------------
def try_parse_datetime(s: pd.Series, dayfirst: bool = True) -> pd.Series:
    """Tenta fazer parse de datas em uma Series (string) -> datetime; onde falhar, NaT."""
    return pd.to_datetime(s, errors="coerce", dayfirst=dayfirst, infer_datetime_format=True)

def looks_like_filename(x: str) -> bool:
    """Heurística simples para detectar se é nome de arquivo/caminho."""
    if not isinstance(x, str):
        return False
    x2 = x.strip().lower()
    return (
        ("/" in x2 or "\\" in x2) or
        (".csv" in x2 or ".txt" in x2 or ".log" in x2 or ".xlsx" in x2)
    )

def to_numeric_coerce(df: pd.DataFrame, skip_cols: list) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if c in skip_cols:
            continue
        out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    """Similaridade do cosseno entre duas séries (ignora NaN por interseção)."""
    mask = ~np.isnan(a) & ~np.isnan(b)
    if mask.sum() == 0:
        return np.nan
    a2, b2 = a[mask], b[mask]
    denom = (np.linalg.norm(a2) * np.linalg.norm(b2))
    return float(np.dot(a2, b2) / denom) if denom else np.nan

def pearson_corr(a: np.ndarray, b: np.ndarray) -> float:
    """Correlação de Pearson entre duas séries (ignora NaN por interseção)."""
    mask = ~np.isnan(a) & ~np.isnan(b)
    if mask.sum() < 3:
        return np.nan
    return float(np.corrcoef(a[mask], b[mask])[0,1])

# ----------------------------
# 1) Leitura (preserva tudo como string inicialmente)
# ----------------------------
df0 = pd.read_csv(INPUT, dtype=str)
df_pre = df0.copy()  # snapshot antes da correção (para relatório de similaridade)

# Padroniza nomes de colunas
df0.columns = [str(c).strip() for c in df0.columns]

if df0.shape[1] < 3:
    raise ValueError("O arquivo precisa ter pelo menos 3 colunas (arquivo|data|... valores).")

col0, col1 = df0.columns[0], df0.columns[1]

# ----------------------------
# 2) Detecta padrões 2023 e 2024 conforme sua descrição
#    - 2023: primeira coluna é data válida
#    - 2024: primeira coluna é nome de arquivo, segunda coluna é data válida
# ----------------------------
date0 = try_parse_datetime(df0[col0])
date1 = try_parse_datetime(df0[col1])

mask_2023_like = date0.notna()  # linha onde primeira coluna é data parseável
mask_2024_shift = (~mask_2023_like) & date1.notna() & df0[col0].apply(looks_like_filename)

# Se tiver ano, refina as máscaras:
#   2023 -> date0.dt.year == 2023
#   2024 -> date1.dt.year == 2024
# (mantém fallback se algum parse não tiver ano)
year0 = date0.dt.year
year1 = date1.dt.year
mask_2023_like = mask_2023_like & ((year0 == 2023) | year0.isna())  # ainda aceita outros anos, mas prioriza 2023
mask_2024_shift = mask_2024_shift & ((year1 == 2024) | year1.isna())

# ----------------------------
# 3) Corrige linhas 2024 deslocadas: "puxa uma coluna à esquerda"
#    Cria 'arquivo_origem' e 'datahora' consistentes
# ----------------------------
df = df0.copy()

# Garante colunas de destino
if "arquivo_origem" not in df.columns:
    df.insert(0, "arquivo_origem", np.nan)
if "datahora" not in df.columns:
    df.insert(1, "datahora", np.nan)

# 3a) Para linhas 2023-like: data na primeira coluna
df.loc[mask_2023_like, "arquivo_origem"] = np.nan
df.loc[mask_2023_like, "datahora"] = df.loc[mask_2023_like, col0]

# 3b) Para linhas 2024-shift: arquivo na col0, data na col1; e o resto deslocado 1 à direita
df.loc[mask_2024_shift, "arquivo_origem"] = df.loc[mask_2024_shift, col0]
df.loc[mask_2024_shift, "datahora"] = df.loc[mask_2024_shift, col1]

# Agora vamos deslocar à esquerda, da col2 até o fim, sobre as mesmas colunas de destino (do col1 em diante),
# preservando nomes de colunas originais de medidas.
orig_cols = df0.columns.tolist()

# Ex.: [C0, C1, C2, C3, ..., Cn]
# Para as linhas 2024-shift, queremos fazer C1 <- C2, C2 <- C3, ..., C(n-1) <- Cn
cols_to_shift_src = orig_cols[2:]        # C2..Cn
cols_to_shift_dst = orig_cols[1:-1]      # C1..C(n-1)

for src, dst in zip(cols_to_shift_src, cols_to_shift_dst):
    df.loc[mask_2024_shift, dst] = df.loc[mask_2024_shift, src]

# Opcional: a última coluna original fica "sobrando" para 2024; podemos limpar
last_col = orig_cols[-1]
df.loc[mask_2024_shift, last_col] = np.nan  # descarta o lixo que "rodou"

# ----------------------------
# 4) Normaliza a coluna datahora para datetime real e ordena por tempo
# ----------------------------
df["datahora"] = try_parse_datetime(df["datahora"])
# Se ainda houver NaT, tenta dayfirst=False como fallback
if df["datahora"].isna().any():
    alt = pd.to_datetime(df["datahora"], errors="coerce", dayfirst=False, infer_datetime_format=True)
    df.loc[df["datahora"].isna(), "datahora"] = alt[df["datahora"].isna()]

# Ordena por timestamp se fizer sentido
if df["datahora"].notna().any():
    df = df.sort_values("datahora").reset_index(drop=True)

# ----------------------------
# 5) Relatório de verificação por similaridade (antes vs depois)
#     - Convertemos colunas numéricas
#     - Comparamos 2023 vs 2024 pré-correção e pós-correção por coluna
# ----------------------------
def build_similarity_report(df_before: pd.DataFrame, df_after: pd.DataFrame) -> pd.DataFrame:
    # Detecta datas antes/depois
    d0_before = try_parse_datetime(df_before[col0])
    d1_before = try_parse_datetime(df_before[col1])
    mask23_before = d0_before.dt.year == 2023
    mask24_before = d1_before.dt.year == 2024  # 2024 estava na segunda coluna

    # Depois da correção, a data está em "datahora"
    d_after = try_parse_datetime(df_after["datahora"])
    mask23_after = d_after.dt.year == 2023
    mask24_after = d_after.dt.year == 2024

    # Seleciona colunas numéricas (exclui arquivo_origem, datahora e as duas primeiras originais)
    skip = {"arquivo_origem", "datahora", col0, col1}
    num_cols_before = [c for c in df_before.columns if c not in skip]
    num_cols_after  = [c for c in df_after.columns if c not in {"arquivo_origem", "datahora"}]

    # Converte para numérico
    b_num = to_numeric_coerce(df_before[num_cols_before], [])
    a_num = to_numeric_coerce(df_after[num_cols_after], [])

    rows = []
    for c in sorted(set(num_cols_before).intersection(num_cols_after)):
        x23_b = b_num.loc[mask23_before, c].astype(float).values
        x24_b = b_num.loc[mask24_before, c].astype(float).values

        x23_a = a_num.loc[mask23_after, c].astype(float).values
        x24_a = a_num.loc[mask24_after, c].astype(float).values

        # Similaridades
        cos_b = cosine_similarity(x23_b, x24_b)
        cor_b = pearson_corr(x23_b, x24_b)

        cos_a = cosine_similarity(x23_a, x24_a)
        cor_a = pearson_corr(x23_a, x24_a)

        rows.append({
            "coluna": c,
            "cos_pre": cos_b, "pearson_pre": cor_b,
            "cos_pos": cos_a, "pearson_pos": cor_a,
            "delta_cos": (cos_a - cos_b) if (pd.notna(cos_a) and pd.notna(cos_b)) else np.nan,
            "delta_pearson": (cor_a - cor_b) if (pd.notna(cor_a) and pd.notna(cor_b)) else np.nan,
        })
    rep = pd.DataFrame(rows).sort_values(["delta_pearson", "delta_cos"], ascending=[False, False])
    return rep

report = build_similarity_report(df_pre, df)

# Mostra um extrato do relatório
print("Relatório de similaridade 2023 vs 2024 (antes vs depois):")
print(report.head(20).to_string(index=False))

# ----------------------------
# 6) Salva resultado curado
# ----------------------------
df.to_csv(OUTPUT, index=False)
print(f"\nArquivo salvo: {OUTPUT.resolve()}")


C:\Users\wilso\AppData\Local\Temp\ipykernel_5136\3362746898.py:20: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(s, errors="coerce", dayfirst=dayfirst, infer_datetime_format=True)
C:\Users\wilso\AppData\Local\Temp\ipykernel_5136\3362746898.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(s, errors="coerce", dayfirst=dayfirst, infer_datetime_format=True)
C:\Users\wilso\AppData\Local\Temp\ipykernel_5136\3362746898.py:20: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pyd

Relatório de similaridade 2023 vs 2024 (antes vs depois):
 coluna  cos_pre  pearson_pre  cos_pos  pearson_pos  delta_cos  delta_pearson
col_003      NaN          NaN      NaN          NaN        NaN            NaN
col_004      NaN          NaN      NaN          NaN        NaN            NaN
col_005      NaN          NaN      NaN          NaN        NaN            NaN
col_006      NaN          NaN      NaN          NaN        NaN            NaN
col_007      NaN          NaN      NaN          NaN        NaN            NaN
col_008      NaN          NaN      NaN          NaN        NaN            NaN
col_009      NaN          NaN      NaN          NaN        NaN            NaN
col_010      NaN          NaN      NaN          NaN        NaN            NaN
col_011      NaN          NaN      NaN          NaN        NaN            NaN
col_012      NaN          NaN      NaN          NaN        NaN            NaN
col_013      NaN          NaN      NaN          NaN        NaN            NaN
col_01

In [5]:
# ============================================================
# Inserir linha 0 com nomes curtos (da ÚLTIMA coluna do mapa)
# após a coluna 'datahora'
# Entradas:
#   - ovation_dados_curado_v1.csv
#   - ovation_mapeado_nome_unidade.csv  (nome curto = última coluna)
# Saída:
#   - ovation_dados_curado_v1_com_nomes.csv
# ============================================================
import pandas as pd
from pathlib import Path

# Caminhos base definidos por você
BASE_DIR = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation")
ARQ_DADOS = BASE_DIR / "ovation_dados_curado_v1.csv"
ARQ_MAPA  = BASE_DIR / "ovation_mapeado_nome_unidade.csv"
ARQ_SAIDA = BASE_DIR / "ovation_dados_curado_v1_com_nomes.csv"

# 1) Leitura como texto para não perder nada
df_dados = pd.read_csv(ARQ_DADOS, dtype=str, low_memory=False)
df_mapa  = pd.read_csv(ARQ_MAPA,  dtype=str)

# 2) Localizar a coluna 'datahora' e quantas colunas de dados existem após ela
cols = df_dados.columns.tolist()
if "datahora" not in cols:
    raise ValueError("Coluna 'datahora' não encontrada em ovation_dados_curado_v1.csv.")
idx_inicio = cols.index("datahora") + 1      # primeira coluna APÓS 'datahora'
n_cols_dados = len(cols) - idx_inicio        # número de colunas a preencher com nomes curtos

# 3) Nome curto = ÚLTIMA coluna do arquivo de mapeamento
col_nome_curto = df_mapa.columns[-1]
nomes_curtos = (
    df_mapa[col_nome_curto]
    .fillna("")
    .astype(str)
    .tolist()
)

# 4) Ajustar quantidade de nomes curtos (completar/truncar)
if len(nomes_curtos) < n_cols_dados:
    faltam = n_cols_dados - len(nomes_curtos)
    nomes_curtos += [f"provisorio_{i+1}" for i in range(faltam)]
elif len(nomes_curtos) > n_cols_dados:
    nomes_curtos = nomes_curtos[:n_cols_dados]

# 5) Construir a nova primeira linha (vazia até 'datahora'; nomes curtos depois)
header_row = [""] * idx_inicio + nomes_curtos

# 6) Inserir a linha 0 e salvar
df_header = pd.DataFrame([header_row], columns=cols)
df_out = pd.concat([df_header, df_dados], ignore_index=True)
df_out.to_csv(ARQ_SAIDA, index=False)

# 7) Relatório rápido
placeholders = [x for x in nomes_curtos if x.startswith("provisorio_")]
print("Resumo da inserção de nomes curtos")
print("-" * 60)
print(f"Arquivo de dados     : {ARQ_DADOS}")
print(f"Arquivo de mapeamento: {ARQ_MAPA}")
print(f"Saída                : {ARQ_SAIDA}")
print(f"Colunas totais       : {len(cols)}")
print(f"Índice pós 'datahora': {idx_inicio} (primeira coluna com nome curto)")
print(f"Qtd. colunas pós 'datahora': {n_cols_dados}")
print(f"Coluna usada como nome curto: '{col_nome_curto}' (última do mapa)")
print(f"Nomes curtos usados  : {len(nomes_curtos)}")
print(f"Placeholders adicionados: {len(placeholders)}")
if placeholders:
    inicio = idx_inicio
    fim = idx_inicio + n_cols_dados
    colunas_pos_datahora = cols[inicio:fim]
    print("\nColunas que receberam placeholders:")
    for i, nome in enumerate(nomes_curtos):
        if nome.startswith("provisorio_"):
            print(f"  - {colunas_pos_datahora[i]}  <-  {nome}")


Resumo da inserção de nomes curtos
------------------------------------------------------------
Arquivo de dados     : C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v1.csv
Arquivo de mapeamento: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_mapeado_nome_unidade.csv
Saída                : C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v1_com_nomes.csv
Colunas totais       : 156
Índice pós 'datahora': 2 (primeira coluna com nome curto)
Qtd. colunas pós 'datahora': 154
Coluna usada como nome curto: 'nome_curto_com_unidade' (última do mapa)
Nomes curtos usados  : 154
Placeholders adicionados: 4

Colunas que receberam placeholders:
  - col_151  <-  provisorio_1
  - col_152  <-  provisorio_2
  - col_153  <-  provisorio_3
  - col_154  <-  provisorio_4


In [6]:
# ============================================================
# Correção de desalinhamento: data foi parar em col_00x
# Alvo: ovation_dados_curado_v1_com_nomes.csv
# Ação:
#   - Se 'datahora' estiver vazia e uma das primeiras colunas de dados
#     (até 3 à frente) contiver uma data, mover essa data para 'datahora'
#     e deslocar os dados à esquerda pelo mesmo número de posições.
#   - Salva de volta no mesmo arquivo, com backup.
# ============================================================
import pandas as pd
import numpy as np
from pathlib import Path
import shutil

BASE_DIR  = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation")
ARQ_ALVO  = BASE_DIR / "ovation_dados_curado_v1_com_nomes.csv"
ARQ_BACK  = BASE_DIR / "ovation_dados_curado_v1_com_nomes.backup.csv"

# 1) Backup
shutil.copy2(ARQ_ALVO, ARQ_BACK)

# 2) Leitura como texto
df = pd.read_csv(ARQ_ALVO, dtype=str, low_memory=False)

cols = df.columns.tolist()
if "datahora" not in cols:
    raise ValueError("Coluna 'datahora' não encontrada no arquivo alvo.")

idx_start = cols.index("datahora") + 1                 # primeira coluna APÓS datahora
data_cols = cols[idx_start:]                           # colunas de dados

# util: testar se string parece data
def is_date_like(x: str) -> bool:
    if not isinstance(x, str) or not x.strip():
        return False
    try:
        # tenta com dayfirst e, se falhar, tenta o padrão
        d1 = pd.to_datetime(x, errors="coerce", dayfirst=True)
        if pd.notna(d1):
            return True
        d2 = pd.to_datetime(x, errors="coerce", dayfirst=False)
        return pd.notna(d2)
    except Exception:
        return False

# 3) Encontrar linhas a ajustar:
#    - datahora vazia/NaT
#    - uma das próximas 3 colunas (col_001, col_002, col_003 relativas) conter data
dh = pd.to_datetime(df["datahora"], errors="coerce", dayfirst=True)
mask_datahora_vazia = dh.isna() | df["datahora"].fillna("").eq("")

# mapeia para índices relativos [0..2] onde possivelmente está a data
MAX_LOOKAHEAD = 3  # quantas colunas à frente procurar
shift_info = np.full(len(df), fill_value=0, dtype=int)
rows_to_fix = []

for i in range(len(df)):
    if not mask_datahora_vazia.iat[i]:
        continue
    # procurar a data nas primeiras N colunas de dados
    found = False
    for rel in range(min(MAX_LOOKAHEAD, len(data_cols))):
        col = data_cols[rel]
        val = df.at[i, col]
        if is_date_like(val):
            shift_info[i] = rel + 1  # deslocamento = quantas posições a data andou a frente (datahora -> col_001 = 1)
            rows_to_fix.append(i)
            found = True
            break
    # se não achou, segue adiante

rows_to_fix = np.array(rows_to_fix, dtype=int)

# 4) Aplicar correção linha a linha:
#    datahora <- valor encontrado
#    para k de 0..(n_cols - shift - 1): data_cols[k] <- data_cols[k + shift]
#    últimos 'shift' campos ficam vazios
ncols_data = len(data_cols)
for i in rows_to_fix:
    shift = shift_info[i]
    # mover data
    df.at[i, "datahora"] = df.at[i, data_cols[shift - 1]]
    # puxar dados à esquerda
    for k in range(ncols_data - shift):
        df.at[i, data_cols[k]] = df.at[i, data_cols[k + shift]]
    # limpar cauda
    for k in range(ncols_data - shift, ncols_data):
        df.at[i, data_cols[k]] = ""

# 5) Salvar de volta (sobrescrevendo o arquivo original)
df.to_csv(ARQ_ALVO, index=False)

# 6) Relatório
print("Correção concluída.")
print(f"Arquivo de entrada : {ARQ_BACK}")
print(f"Arquivo corrigido  : {ARQ_ALVO}")
print(f"Linhas avaliadas   : {len(df)}")
print(f"Linhas corrigidas  : {len(rows_to_fix)}")
if len(rows_to_fix):
    print(f"Exemplo de linhas corrigidas (primeiras 10): {rows_to_fix[:10].tolist()}")


Correção concluída.
Arquivo de entrada : C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v1_com_nomes.backup.csv
Arquivo corrigido  : C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v1_com_nomes.csv
Linhas avaliadas   : 21841
Linhas corrigidas  : 0


In [7]:
# ============================================================
# Realinhar a linha 0 de nomes-curtos para começar na 4ª coluna
# (colunas 1 e 2 vazias; 3ª é datahora; nomes curtos a partir da 4ª)
# Também higieniza a coluna 'datahora' removendo aspas extras.
# Alvo: ovation_dados_curado_v1_com_nomes.csv
# ============================================================
import pandas as pd
from pathlib import Path
import shutil

BASE_DIR  = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation")
ARQ_ALVO  = BASE_DIR / "ovation_dados_curado_v1_com_nomes.csv"
ARQ_BACK  = BASE_DIR / "ovation_dados_curado_v1_com_nomes.before_realign.csv"

# 1) Backup
shutil.copy2(ARQ_ALVO, ARQ_BACK)

# 2) Ler como texto
df = pd.read_csv(ARQ_ALVO, dtype=str, low_memory=False)

cols = df.columns.tolist()
ncol = len(cols)

# Sanidade: garantir que exista 'datahora' como 3ª coluna (idx 2)
# (Se o cabeçalho não estiver assim, apenas seguimos, pois vamos sobrescrever a linha 0)
# 3) Recriar a linha 0 usando o arquivo de mapeamento (última coluna = nome curto)
MAPA = BASE_DIR / "ovation_mapeado_nome_unidade.csv"
df_mapa = pd.read_csv(MAPA, dtype=str)

# Nomes curtos = última coluna do mapa
col_nome_curto = df_mapa.columns[-1]
nomes = df_mapa[col_nome_curto].fillna("").astype(str).tolist()

# Precisamos preencher todas as colunas APÓS a 3ª (0-based: 3..ncol-1)
n_pos_datahora = ncol - 3
if len(nomes) < n_pos_datahora:
    # completar com provisorios
    faltam = n_pos_datahora - len(nomes)
    nomes = nomes + [f"provisorio_{i+1}" for i in range(faltam)]
elif len(nomes) > n_pos_datahora:
    nomes = nomes[:n_pos_datahora]

# Nova linha 0: col0="", col1="", col2="", col3.. = nomes curtos
linha0 = ["", "", ""] + nomes

# 4) Substituir a linha 0 do DataFrame
df.iloc[0] = linha0

# 5) Higienizar 'datahora': remover aspas extras
if "datahora" in df.columns:
    df["datahora"] = df["datahora"].astype(str).str.replace(r'^"+|"+$', '', regex=True)

# 6) Salvar de volta
df.to_csv(ARQ_ALVO, index=False, encoding="utf-8", lineterminator="\n")

print("Realinhamento concluído.")
print(f"Backup             : {ARQ_BACK}")
print(f"Arquivo corrigido  : {ARQ_ALVO}")
print(f"Colunas totais     : {ncol}")
print(f"Nomes curtos usados: {len(nomes)} (a partir da 4ª coluna)")


Realinhamento concluído.
Backup             : C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v1_com_nomes.before_realign.csv
Arquivo corrigido  : C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v1_com_nomes.csv
Colunas totais     : 156
Nomes curtos usados: 153 (a partir da 4ª coluna)


In [8]:
# ============================================================
# Realinhar a linha 0 de nomes-curtos para começar na 4ª coluna
# e verificar visualmente (transposição das 5 primeiras linhas).
# ============================================================
import pandas as pd
from pathlib import Path
import shutil

BASE_DIR  = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation")
ARQ_ALVO  = BASE_DIR / "ovation_dados_curado_v1_com_nomes.csv"
ARQ_BACK  = BASE_DIR / "ovation_dados_curado_v1_com_nomes.before_realign.csv"

# 1) Backup
shutil.copy2(ARQ_ALVO, ARQ_BACK)

# 2) Ler como texto
df = pd.read_csv(ARQ_ALVO, dtype=str, low_memory=False)
ncol = len(df.columns)

# 3) Recriar a linha 0 usando o arquivo de mapeamento
MAPA = BASE_DIR / "ovation_mapeado_nome_unidade.csv"
df_mapa = pd.read_csv(MAPA, dtype=str)

# Nomes curtos = última coluna do mapa
col_nome_curto = df_mapa.columns[-1]
nomes = df_mapa[col_nome_curto].fillna("").astype(str).tolist()

# Quantas colunas devem receber nome curto
n_pos_datahora = ncol - 3
if len(nomes) < n_pos_datahora:
    nomes += [f"provisorio_{i+1}" for i in range(n_pos_datahora - len(nomes))]
elif len(nomes) > n_pos_datahora:
    nomes = nomes[:n_pos_datahora]

# Nova linha 0
linha0 = ["", "", ""] + nomes
df.iloc[0] = linha0

# 4) Higienizar 'datahora': remover aspas extras
if "datahora" in df.columns:
    df["datahora"] = df["datahora"].astype(str).str.replace(r'^"+|"+$', '', regex=True)

# 5) Salvar
df.to_csv(ARQ_ALVO, index=False, encoding="utf-8", lineterminator="\n")

print("Realinhamento concluído.")
print(f"Backup             : {ARQ_BACK}")
print(f"Arquivo corrigido  : {ARQ_ALVO}")
print(f"Nomes curtos usados: {len(nomes)} (a partir da 4ª coluna)")

# 6) Verificação: transpor as 5 primeiras linhas
amostra_transposta = df.head(5).transpose().reset_index()
amostra_transposta.columns = ["variavel"] + [f"Linha_{i}" for i in range(1, 6)]
print("\n=== Amostra transposta das 5 primeiras linhas ===")
print(amostra_transposta.to_string(index=False))


Realinhamento concluído.
Backup             : C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v1_com_nomes.before_realign.csv
Arquivo corrigido  : C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v1_com_nomes.csv
Nomes curtos usados: 153 (a partir da 4ª coluna)

=== Amostra transposta das 5 primeiras linhas ===
      variavel                               Linha_1              Linha_2              Linha_3              Linha_4              Linha_5
arquivo_origem                                                        NaN                  NaN                  NaN                  NaN
      datahora                                                        nan                  nan                  nan                  nan
       col_001                                       "06/29/2023 23:00:00 "06/29/2023 22:00:00 "06/29/2023 21:00:00 "06/29/2023 20:00:00
       col_002    flw_output_of_belt_coal_fdr_a1_t_h        

In [9]:
# ============================================================
# Realinhar nomes curtos:
#  - Linha 0: começar na 3ª coluna (1-based)
#  - Coluna 3 = "TIMESTAMP"
#  - A partir da 4ª: nomes curtos do mapa (deslocados em 1)
#  - Adicionar um espaço no nome curto das colunas originais col_012 e col_014
#  - Imprimir as 10 primeiras linhas transpostas
# ============================================================
import pandas as pd
from pathlib import Path
import shutil

BASE_DIR  = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation")
ARQ_ALVO  = BASE_DIR / "ovation_dados_curado_v1_com_nomes.csv"
ARQ_MAPA  = BASE_DIR / "ovation_mapeado_nome_unidade.csv"
ARQ_BACK  = BASE_DIR / "ovation_dados_curado_v1_com_nomes.before_shift.csv"

# 1) Backup do arquivo alvo
shutil.copy2(ARQ_ALVO, ARQ_BACK)

# 2) Leitura
df = pd.read_csv(ARQ_ALVO, dtype=str, low_memory=False)
df_map = pd.read_csv(ARQ_MAPA, dtype=str)

cols = df.columns.tolist()
ncol = len(cols)

# 3) Extrair nomes curtos do mapa (última coluna)
col_nome_curto = df_map.columns[-1]
nomes = df_map[col_nome_curto].fillna("").astype(str).tolist()

# 4) Linha 0 desejada:
#    col1="", col2="", col3="TIMESTAMP", col4.. = nomes curtos (ajustar tamanho)
n_after_col3 = ncol - 3  # quantidade de posições a preencher a partir da 4ª coluna
if len(nomes) < n_after_col3:
    # completar com placeholders se faltarem nomes
    nomes += [f"provisorio_{i+1}" for i in range(n_after_col3 - len(nomes))]
elif len(nomes) > n_after_col3:
    # truncar se sobrar
    nomes = nomes[:n_after_col3]

linha0 = ["", "", "TIMESTAMP"] + nomes

# 5) Aplicar linha 0 no DataFrame
df.iloc[0] = linha0

# 6) Adicionar um espaço no nome curto das colunas originais col_012 e col_014
for col_fix in ["col_012", "col_014"]:
    if col_fix in cols:
        idx = cols.index(col_fix)
        df.iat[0, idx] = " " + (df.iat[0, idx] if pd.notna(df.iat[0, idx]) else "")

# 7) Limpar aspas extras em datahora (se houver)
if "datahora" in df.columns:
    df["datahora"] = df["datahora"].astype(str).str.replace(r'^"+|"+$', '', regex=True)

# 8) Salvar
df.to_csv(ARQ_ALVO, index=False, encoding="utf-8", lineterminator="\n")

# 9) Verificação: imprimir as 10 primeiras linhas TRANSPOSTAS
amostra = df.head(10).transpose().reset_index()
amostra.columns = ["variavel"] + [f"Linha_{i}" for i in range(1, len(amostra.columns))]
print(amostra.to_string(index=False))
print("\nOK: nomes curtos iniciados na 3ª coluna (TIMESTAMP) e shift aplicado.")
print(f"Backup em: {ARQ_BACK}")
print(f"Arquivo salvo: {ARQ_ALVO}")


      variavel                               Linha_1              Linha_2              Linha_3              Linha_4              Linha_5              Linha_6              Linha_7              Linha_8              Linha_9             Linha_10
arquivo_origem                                                        NaN                  NaN                  NaN                  NaN                  NaN                  NaN                  NaN                  NaN                  NaN
      datahora                                                        nan                  nan                  nan                  nan                  nan                  nan                  nan                  nan                  nan
       col_001                             TIMESTAMP "06/29/2023 23:00:00 "06/29/2023 22:00:00 "06/29/2023 21:00:00 "06/29/2023 20:00:00 "06/29/2023 19:00:00 "06/29/2023 18:00:00 "06/29/2023 17:00:00 "06/29/2023 16:00:00 "06/29/2023 15:00:00
       col_002    flw_output_of_

In [10]:
import pandas as pd
from pathlib import Path
import shutil

# Caminho do arquivo
ARQ = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v2_com_nomes.csv")
BACKUP = ARQ.with_suffix(".before_remove_firstline.csv")

# 1) Backup antes de alterar
shutil.copy2(ARQ, BACKUP)

# 2) Leitura
df = pd.read_csv(ARQ, low_memory=False)

# 3) Remover a primeira linha
df = df.iloc[1:].reset_index(drop=True)

# 4) Salvar sobrescrevendo o original
df.to_csv(ARQ, index=False, encoding="utf-8", lineterminator="\n")

print(f"Primeira linha removida.")
print(f"Backup salvo em: {BACKUP}")
print(f"Arquivo sobrescrito: {ARQ}")


Primeira linha removida.
Backup salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v2_com_nomes.before_remove_firstline.csv
Arquivo sobrescrito: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v2_com_nomes.csv


In [11]:
import pandas as pd
from pathlib import Path
import shutil

# Caminho do arquivo
ARQ = Path(r"C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v2_com_nomes.csv")
BACKUP = ARQ.with_suffix(".before_set_header.csv")

# 1) Backup
shutil.copy2(ARQ, BACKUP)

# 2) Leitura sem header automático
df = pd.read_csv(ARQ, header=None, low_memory=False)

# 3) Obter a segunda linha como header
new_header = df.iloc[1].tolist()

# 4) Remover as duas primeiras linhas e aplicar o novo header
df = df.iloc[2:].reset_index(drop=True)
df.columns = new_header

# 5) Salvar sobrescrevendo
df.to_csv(ARQ, index=False, encoding="utf-8", lineterminator="\n")

print(f"Cabeçalho substituído e primeiras linhas removidas.")
print(f"Backup salvo em: {BACKUP}")
print(f"Arquivo sobrescrito: {ARQ}")

# 6) Mostrar as 5 primeiras linhas para conferência
print(df.head())


Cabeçalho substituído e primeiras linhas removidas.
Backup salvo em: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v2_com_nomes.before_set_header.csv
Arquivo sobrescrito: C:\Users\wilso\MBA_EMPREENDEDORISMO\3AGD\A1_LOCAL\data\raw\ovation\ovation_dados_curado_v2_com_nomes.csv
  Timestamp;;flw_output_of_belt_coal_fdr_a1_t_h;flw_output_of_belt_coal_fdr_a2_t_h;flw_output_of_belt_coal_fdr_b1_t_h;flw_output_of_belt_coal_fdr_b2_t_h;flw_output_of_belt_coal_fdr_b3_t_h;flw_output_of_belt_coal_fdr_b4_t_h;flw_output_of_belt_coal_fdr_c1_t_h;flw_output_of_belt_coal_fdr_c2_t_h;porcentagem_de_ar_primario_;upper_perc_do_ar_total_;burner_perc_do_ar_total_;lower_perc_do_ar_total_;coal_perc_do_ar_total_;temp_1_of_furnace_a_inner_adegc;pressure_of_furnace_a_lower_kpa;pressure_1_of_furnace_a_bed_kpa;pressure_3_of_furnace_bed_kpa;press_2_3_of_furnace_b_bed_kpa;pressure_1_of_furnace_b_bed_kpa;pressure_of_furnace_b_lower_kpa;temp_1_of_furnace_b_inner_adegc;furnace_bed_